# Сеточный мир

## Постановка задачи

Условия: задана матрица состояний 5х5, функция ценности перехода из одного состояния в другое, $\gamma = 0.9$. Стратегия выбора перехода (действия) - равновероятная.

Задача: для каждого состояния вычислить его ценность. Вычисления провести двумя методами: путём решения СЛАУ и методом Монте-Карло.

<img src="mesh_world_problem.png" width=600 height=600/>

## Решение

### Монте-Карло

Определим функцию, которая делает броски по Монте-Карло и для перехода из одного состояния в другое пользуется предоставленной стратегией.

Стратегия - это функция, которая принимает на вход текущее состояние и возвращает следующее состояние и награду за переход в следующее состояние.

In [73]:
from typing import Callable, Tuple

import numpy as np

def mc_solution(
        strategy: Callable[[int, int], Tuple[int, int, float]],
        width: int=5, 
        height: int=5, 
        gamma: float=0.9, 
        exp_cnt: int=10**6
    ):
    '''
    Monte-Carlo based solution. Model `exp_cnt` experiments, extract average out of results.
    Arguments:
    * width, height - size of statuses' matrix.
    * gamma - decay coefficient.
    * exp_cnt - number of experiments per state.
    * strategy - callable object, which accepts current state (starting from (1,1)) and returns next state (also starting from (1,1))
and reward for passing to this state.
    
    Returns numpy.ndarray of size height*width, which elements are value function of corresponding state.
    '''
    def _make_throw(i, j, gamma, gamma_i = -1):
        eps = 10**(-20)
        result_i = 0.
        gamma_i = gamma if gamma_i == -1 else gamma_i
        
        next_i, next_j, reward = strategy(i, j)
        result_i += reward
        if gamma_i >= eps:
            result_i += gamma_i * _make_throw(next_i, next_j, gamma, gamma_i*gamma)
        
        return result_i
    
    result = np.zeros((height, width))
    
    for i in range(height):
        for j in range(width):
            total_value = 0.
            
            for _ in range(exp_cnt):
                total_value += _make_throw(i+1, j+1, gamma)
            result[i,j] = total_value / exp_cnt
            
    return result

Определим стратегию, которая соответствует нашей задаче.

In [61]:
import numpy as np

def strategy(i: int, j: int):
    possible_states = [
        (i+1, j),
        (i-1, j),
        (i, j+1),
        (i, j-1)
    ]
    reward = 0.
    
    next_state = possible_states[np.random.randint(0, len(possible_states))]
    
    # Check for special states.
    if i == 1 and j == 2:
        next_state = (5,2)
        reward = 10.
    elif i == 1 and j == 4:
        next_state = (3,4)
        reward = 5.
        
    # Check if we are out of grid.
    if next_state[0] < 1 or next_state[0] > 5 or \
       next_state[1] < 1 or next_state[1] > 5:
        next_state = (i,j)
        reward = -1.
    
    return *next_state, reward

In [70]:
print(mc_solution(strategy, exp_cnt=100))

[[ 2.59008863  9.29338037  4.3727448   5.15254297  0.92513858]
 [ 0.5995925   2.69892745  1.21601399  1.21520269  0.08390912]
 [-0.28401662  0.8429217   0.55434667  0.12485087 -0.53017915]
 [-0.76850065 -0.3020796  -0.31179786 -0.30295974 -1.00429402]
 [-1.35037959 -0.87185098 -0.90029531 -1.02843573 -1.36728573]]


In [74]:
print(mc_solution(strategy, exp_cnt=1000))

[[ 2.92336535  9.33980491  4.04449744  5.07761607  0.87255929]
 [ 0.95962867  2.59067479  1.5676385   1.30325653  0.04143557]
 [-0.32747972  0.44869587  0.31706527  0.17354382 -0.53247775]
 [-0.7740485  -0.28153534 -0.20158665 -0.335326   -0.82808145]
 [-1.32019219 -0.91071728 -0.82738497 -0.8842017  -1.38595556]]


In [72]:
print(mc_solution(strategy, exp_cnt=10000))

[[ 2.7965825   9.3296535   4.09893173  5.09685239  0.93112314]
 [ 0.77731217  2.63244149  1.67177777  1.41307513  0.03839509]
 [-0.29137979  0.44934179  0.34575188  0.14816086 -0.51668063]
 [-0.80676709 -0.29933235 -0.20522902 -0.35632894 -0.8485803 ]
 [-1.37335368 -0.87542537 -0.76551391 -0.90011059 -1.37683771]]


### Решение СЛАУ

Мы знаем следующее: $\nu_{\pi}(s) = \sum_{a}{\pi(a|s)}\sum_{s'}\sum_{r}p(s',r|s,a)[r + \gamma\nu_{\pi}(s')]$

Наша стратегия - равновероятная, поэтому $\pi(a|s) = 1/4$ для обычных клеток (всего четыре различных действия), $\pi(a|s) = 1$ для клеток A и B (в них мы можем сделать только одно конкретное действие).

Также нам известно, что при любом действии у нас возможно только одно вознаграждение и переход в одно конкретное состояние.

Таким образом, изначальная формула преобразуется в 

$\nu_{\pi}(s) = \begin{cases} \sum_{a}\frac{1}{4}(r_{a} + \gamma\nu_{\pi}(s'_{a})), & s \notin \{A,B\} \\ r + \gamma\nu_{\pi}(s'), & s \in \{A,B\} \end{cases}$

Подставив в формулу каждое состояние $s$, получим СЛАУ, решив которую, мы получим искомые стоимости.

Формула выше в матричном виде будет иметь вид $Ax + b = x$, 

где $b = (\sum_{a_{s_{1}}}\frac{1}{4}r_{a_{s_{1}}} \dots \sum_{a_{s_{n}}}\frac{1}{4}r_{a_{s_{n}}})^{T}$ - вектор свободных членов, которые являются матожиданием награды в соответствующих состояниях,

$a_{ij} = \begin{cases} \sum{\frac{\gamma}{4}}, & \mbox{если из состояния } s_{i} \mbox{ можно перейти в состояние } s_{j} \mbox{ и } s_{i} \notin \{A,B\} \mbox{. Сумма считается по всем действиям, в результате которых из состояния } s_{i} \mbox{ достигается состояние } s_{j}, \\ \gamma, & \mbox{если из состояния } s_{i} \mbox{ можно перейти в состояние } s_{j} \mbox{ и } s_{i} \in \{A,B\}, \\ 0, & \mbox{иначе.} \end{cases}$,

$x$ - вектор $\nu_{\pi}(s)$.

Преобразовав это уравнение, получим $(A-E)x = -b$, 

оно же $(E - A)x = b$.

In [128]:
from typing import Callable, Generator, Tuple

import numpy as np

def system_solution(
        action_generator: Callable[[int, int], Generator[Tuple[int, int, float], None, None]],
        n: int=5,
        m: int=5,
        gamma: float=0.9
    ):
    n,m = 5,5
    a = np.eye(n*m)
    b = np.zeros(n*m)
    
    for i in range(n):
        for j in range(m):
            current_state = i*m + j
            states_to_update = set()
            
            for state_info in action_generator(i+1, j+1):
                next_state = (state_info[0]-1)*m + (state_info[1]-1)
                reward = state_info[2]
                
                states_to_update.add(next_state)
                b[current_state] += reward
                
            states_cnt = len(states_to_update)
            for state in states_to_update:
                a[current_state, state] -= gamma / states_cnt
    
    result = np.linalg.solve(a, b)
    result.resize((n,m))
    
    return result

In [129]:
def action_generator(i: int, j: int):
    possible_states = [
        (i+1, j),
        (i-1, j),
        (i, j+1),
        (i, j-1)
    ]
    
    if i == 1 and j == 2:
        yield 5, 2, 10.
    elif i == 1 and j == 4:
        yield 3, 4, 5.
    else:
        for state in possible_states:
            reward = 0.
            
            if state[0] < 1 or state[0] > 5 or \
               state[1] < 1 or state[1] > 5:
                state = (i,j)
                reward = -1. / 4
                
            yield *state, reward
        

In [130]:
print(system_solution(action_generator))

[[ 3.86436159  8.9207585   4.51751998  5.40817026  1.91987469]
 [ 1.76275187  3.12105378  2.34252893  2.01204579  0.73820401]
 [ 0.19739664  0.84531084  0.76062014  0.45352252 -0.27810667]
 [-0.81702983 -0.32213352 -0.26082831 -0.47890363 -1.03853838]
 [-1.57836588 -1.19915722 -1.11881992 -1.28261642 -1.70906634]]
